## Import the necessary modules

In [1]:
import os
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import logging

## Create a logger

In [2]:
logger = logging.getLogger("[Database Structures and Management with MySQL]")
if os.path.exists("log/management-with-mysql.log"):
  os.remove("log/management-with-mysql.log")
logging.basicConfig(filename='log/management-with-mysql.log', encoding='utf-8', level=logging.DEBUG, format='%(asctime)s ==> %(message)s', datefmt='%m/%d/%Y %I:%M:%S')

## Establish a connection using MySQlConnectionPool 

In [3]:
dbconfig={"user":"root", "password":os.environ["MYSQL_PASSWORD"], "port":33061, "host":"localhost"}
try:
    logger.info("Creating connection pools")
    pool = MySQLConnectionPool(pool_size=4, pool_name="pool_b", **dbconfig)
    print(f"The connection pool is created with a name: {pool.pool_name}")
    logger.info(f"The connection pool is created with a name: {pool.pool_name}")
    print(f"The pool size is: {pool.pool_size}")
    logger.info(f"The pool size is: {pool.pool_size}")
except Error as err:
    print(f"Error code: {err.errno}")
    logger.info(f"Error code: {err.errno}")
    print(f"Error message: {err.msg}")
    logger.info(f"Error message: {err.msg}")

The connection pool is created with a name: pool_b
The pool size is: 4


## Creating a connection from the connection pool

In [4]:
first_connection = pool.get_connection()
print("First connections established between MySQL and Python")
logger.info("First connections established between MySQL and Python")

First connections established between MySQL and Python


## Create a cursor objects

In [5]:
print("Creating first cursor object from the first connection")
logger.info("Creating first cursor object from the first connection")
first_cursor = first_connection.cursor(buffered=True)
print("First Cursor object created to communicate with MySQL using Python.")
logger.info("First Cursor object created to communicate with MySQL using Python.")

Creating first cursor object from the first connection
First Cursor object created to communicate with MySQL using Python.


## Create Database 

In [6]:
# Get a cursor object from the cursor pool
database_name = "db_meta_dsm_mysql"
drop_database_query = f"""DROP DATABASE IF EXISTS {database_name}"""
first_cursor.execute(drop_database_query)
logger.info("Dropping Database if it already exists.")

create_database_query: str = f"""CREATE DATABASE IF NOT EXISTS {database_name}"""
print("Creating Database.")
logger.info("Creating Database.")
first_cursor.execute(create_database_query)
logger.info("Database created.")
print("Database created.")

Creating Database.
Database created.


In [7]:
# Check to see that the database was created
list_of_databases: list = []
first_cursor.execute("SHOW DATABASES;")
databases = first_cursor.fetchall()
for database in databases:
    db_name: str = database[0]
    list_of_databases.append(db_name)
    if db_name == database_name:
        print(f"Database '{database_name}' was successfully created")
        logger.info(f"Database '{database_name}' was successfully created.")
        break
print(list_of_databases)
logger.info(list_of_databases)

Database 'db_meta_dsm_mysql' was successfully created
['CVD', 'LittleLemonDB', 'PC', 'STAFF_LOCATIONS', 'classicmodels', 'db_Exercise', 'db_hr', 'db_learner', 'db_little_lemon', 'db_meta', 'db_meta_advanced_mysql', 'db_meta_dsm_mysql']


In [8]:
# Set the new created database as the database to use
first_cursor.execute(f"USE {database_name}")
print(f"Database '{database_name}' is set for use.")
logger.info(f"Database '{database_name}' is set for use.")

Database 'db_meta_dsm_mysql' is set for use.


## Assert Database contains no Tables

In [9]:
## Expect an empty list to be returned 
show_tables_query = """SHOW TABLES;"""
first_cursor.execute(show_tables_query)
results = first_cursor.rowcount
assert results == 0

## Create Table

In [10]:
create_table_query = """CREATE TABLE IF NOT EXISTS tbl_orders (OrderID INT NOT NULL PRIMARY KEY, ClientID VARCHAR(10),  ProductID VARCHAR(10),  Quantity INT, Cost DECIMAL(6,2));"""
first_cursor.execute(create_table_query)
print("Table tbl_orders successfully created")
logger.info("Table tbl_orders successfully created")

Table tbl_orders successfully created


## Populate the table with data

In [11]:
insert_into_tbl_order_query = """INSERT INTO tbl_orders (OrderID, ClientID, ProductID , Quantity, Cost) VALUES (1, "Cl1", "P1", 10, 500), (2, "Cl2", "P2", 5, 100), (3, "Cl3", "P3", 20, 800), (4, "Cl4", "P4", 15, 150),
(5, "Cl3", "P3", 10, 450), (6, "Cl2", "P2", 5, 800), (7, "Cl1", "P4", 22, 1200), (8, "Cl3", "P1", 15, 150), (9, "Cl1", "P1", 10, 500), (10, "Cl2", "P2", 5, 100);"""

# Populate Menu table
print("Inserting records into the tbl_orders table")
logger.info("Inserting records into the tbl_orders table")
first_cursor.execute(insert_into_tbl_order_query)
first_connection.commit()

Inserting records into the tbl_orders table


## Create a function to display results

In [12]:
def select_all_query(table_name: str):
    return f"""SELECT * FROM {table_name};"""


def display_results(table_column_names: list, results: list):
    table_columns_length = [len(x) for x in table_column_names]
    for result in results:
        for value in range(len(result)):
            row_data = result[value]
            if row_data:
                row_data = str(row_data)
                if len(row_data) > table_columns_length[value]:
                    table_columns_length[value] = len(row_data)
    dashes_plus = ""
    for num in range(len(table_columns_length)):
        dashes_plus = dashes_plus + "+" + '-'*(table_columns_length[num]+2)
    dashes_plus = dashes_plus + "+"
    
    print(dashes_plus)
    
    table_headers = ""
    for num in range(len(table_column_names)):
        table_headers = table_headers + f"| {table_column_names[num]:^{table_columns_length[num]}} "
    table_headers = table_headers + "|"
    print(table_headers)
    
    print(dashes_plus)
    
    for result in results:
        table_row = ""
        for value in range(len(result)):
            row_data = result[value]
            if not row_data:
                if "Field" in table_column_names:
                    row_data = " NULL"
                else:
                    row_data = "None"
            table_row = table_row + "|" + f"{str(row_data):^{table_columns_length[value]+2}}"
        print(table_row + "|")
    print(dashes_plus)

def execute_display_query_results(query: str = "", table_column_names: list = [], results: list = []): 
    logger.info(f"Executing the query: {query}")
    if len(query) > 2 and (table_column_names or results):
        print("You can only pass in the query alone or the table_column_names and results list")
        assert False
    if query and not table_column_names and not results:
        first_cursor.execute(query)
        results = first_cursor.fetchall()    
        table_column_names = first_cursor.column_names
    
    display_results(table_column_names, results)

In [13]:
## Show columns for table tbl_orders
show_query = "SHOW COLUMNS FROM tbl_orders;"
execute_display_query_results(show_query)

+-----------+--------------+------+-----+---------+-------+
|   Field   |     Type     | Null | Key | Default | Extra |
+-----------+--------------+------+-----+---------+-------+
|  OrderID  |     int      |  NO  | PRI |   NULL  |  NULL |
| ClientID  | varchar(10)  | YES  | NULL|   NULL  |  NULL |
| ProductID | varchar(10)  | YES  | NULL|   NULL  |  NULL |
| Quantity  |     int      | YES  | NULL|   NULL  |  NULL |
|   Cost    | decimal(6,2) | YES  | NULL|   NULL  |  NULL |
+-----------+--------------+------+-----+---------+-------+


In [14]:
## Show all the records in the table
execute_display_query_results(select_all_query("tbl_orders"))

+---------+----------+-----------+----------+---------+
| OrderID | ClientID | ProductID | Quantity |  Cost   |
+---------+----------+-----------+----------+---------+
|    1    |   Cl1    |    P1     |    10    | 500.00  |
|    2    |   Cl2    |    P2     |    5     | 100.00  |
|    3    |   Cl3    |    P3     |    20    | 800.00  |
|    4    |   Cl4    |    P4     |    15    | 150.00  |
|    5    |   Cl3    |    P3     |    10    | 450.00  |
|    6    |   Cl2    |    P2     |    5     | 800.00  |
|    7    |   Cl1    |    P4     |    22    | 1200.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00  |
|    9    |   Cl1    |    P1     |    10    | 500.00  |
|   10    |   Cl2    |    P2     |    5     | 100.00  |
+---------+----------+-----------+----------+---------+


# Filtering Task

## Task 1
Write a SQL statement to print all records of orders where the cost is $250 or less.

In [15]:
select_query = "SELECT * FROM tbl_orders WHERE Cost <= 250;"
execute_display_query_results(select_query)

+---------+----------+-----------+----------+--------+
| OrderID | ClientID | ProductID | Quantity |  Cost  |
+---------+----------+-----------+----------+--------+
|    2    |   Cl2    |    P2     |    5     | 100.00 |
|    4    |   Cl4    |    P4     |    15    | 150.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00 |
|   10    |   Cl2    |    P2     |    5     | 100.00 |
+---------+----------+-----------+----------+--------+


## Task 2
Write a SQL statement to print all records of orders where the cost is between \\$50 and \\$750.

In [16]:
select_query = "SELECT * FROM tbl_orders WHERE Cost BETWEEN 50 and 750;"
execute_display_query_results(select_query)

+---------+----------+-----------+----------+--------+
| OrderID | ClientID | ProductID | Quantity |  Cost  |
+---------+----------+-----------+----------+--------+
|    1    |   Cl1    |    P1     |    10    | 500.00 |
|    2    |   Cl2    |    P2     |    5     | 100.00 |
|    4    |   Cl4    |    P4     |    15    | 150.00 |
|    5    |   Cl3    |    P3     |    10    | 450.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00 |
|    9    |   Cl1    |    P1     |    10    | 500.00 |
|   10    |   Cl2    |    P2     |    5     | 100.00 |
+---------+----------+-----------+----------+--------+


## Task 3
Write a SQL statement to print all records of orders that have been placed by the client with the id of Cl3 and where the cost of the order is more than $100.

In [17]:
select_query = "SELECT * FROM tbl_orders WHERE ClientID = 'Cl3' AND Cost > 100;"
execute_display_query_results(select_query)

+---------+----------+-----------+----------+--------+
| OrderID | ClientID | ProductID | Quantity |  Cost  |
+---------+----------+-----------+----------+--------+
|    3    |   Cl3    |    P3     |    20    | 800.00 |
|    5    |   Cl3    |    P3     |    10    | 450.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00 |
+---------+----------+-----------+----------+--------+


## Task 4
Write a SQL statement to print all records of orders that have a product id of p1 or p2 and the order quantity is more than 2. 

In [18]:
select_query = "SELECT * FROM tbl_orders WHERE ProductID IN ('P1', 'P2') AND Quantity > 2;"
execute_display_query_results(select_query)

+---------+----------+-----------+----------+--------+
| OrderID | ClientID | ProductID | Quantity |  Cost  |
+---------+----------+-----------+----------+--------+
|    1    |   Cl1    |    P1     |    10    | 500.00 |
|    2    |   Cl2    |    P2     |    5     | 100.00 |
|    6    |   Cl2    |    P2     |    5     | 800.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00 |
|    9    |   Cl1    |    P1     |    10    | 500.00 |
|   10    |   Cl2    |    P2     |    5     | 100.00 |
+---------+----------+-----------+----------+--------+


# Create Additional Tables

In [19]:
## Create additional tables
create_table_customers_query = """CREATE TABLE IF NOT EXISTS tbl_customers(CustomerID INT NOT NULL PRIMARY KEY, FullName VARCHAR(100) NOT NULL, PhoneNumber INT NOT NULL UNIQUE);"""

create_table_bookings_query = """CREATE TABLE IF NOT EXISTS tbl_bookings (BookingID INT NOT NULL PRIMARY KEY,  BookingDate DATE NOT NULL, TableNumber INT NOT NULL, NumberOfGuests INT NOT NULL CHECK (NumberOfGuests <= 8), 
CustomerID INT NOT NULL, FOREIGN KEY (CustomerID) REFERENCES tbl_customers (CustomerID) ON DELETE CASCADE ON UPDATE CASCADE);"""

create_table_courses_query = """CREATE TABLE IF NOT EXISTS tbl_courses (CourseName VARCHAR(255) PRIMARY KEY, Cost Decimal(4,2));"""


first_cursor.execute(create_table_customers_query)
print("Table tbl_customers successfully created")
logger.info("Table tbl_customers successfully created")

first_cursor.execute(create_table_bookings_query)
print("Table tbl_bookings successfully created")
logger.info("Table tbl_bookings successfully created")

first_cursor.execute(create_table_courses_query)
print("Table tbl_courses successfully created")
logger.info("Table tbl_courses successfully created")

Table tbl_customers successfully created
Table tbl_bookings successfully created
Table tbl_courses successfully created


In [20]:
## Populate the tables with Data
insert_into_tbl_customers = """INSERT INTO tbl_customers(CustomerID, FullName, PhoneNumber) VALUES 
(1, "Vanessa McCarthy", 0757536378), (2, "Marcos Romero", 0757536379), (3, "Hiroki Yamane", 0757536376), (4, "Anna Iversen", 0757536375), (5, "Diana Pinto", 0757536374),     
(6, "Altay Ayhan", 0757636378), (7, "Jane Murphy", 0753536379), (8, "Laurina Delgado", 0754536376), (9, "Mike Edwards", 0757236375), (10, "Karl Pederson", 0757936374);"""
logger.info("Inserting records into the tbl_customers table")
first_cursor.execute(insert_into_tbl_customers)
first_connection.commit()

insert_into_tbl_bookings = """INSERT INTO tbl_bookings (BookingID, BookingDate, TableNumber, NumberOfGuests, CustomerID) VALUES (10, '2021-11-10', 7, 5, 1), (11, '2021-11-10', 5, 2, 2),  
(12, '2021-11-10', 3, 2, 4), (13, '2021-11-11', 2, 5, 5),  (14, '2021-11-11', 5, 2, 6), (15, '2021-11-11', 3, 2, 7), (16, '2021-11-11', 3, 5, 1),  (17, '2021-11-12', 5, 2, 2),  
(18, '2021-11-12', 3, 2, 4), (19, '2021-11-13', 7, 5, 6),  (20, '2021-11-14', 5, 2, 3), (21, '2021-11-14', 3, 2, 4);"""
logger.info("Inserting records into the tbl_bookings table")
first_cursor.execute(insert_into_tbl_bookings)
first_connection.commit()

insert_into_tbl_courses = """INSERT INTO tbl_courses (CourseName, Cost) VALUES ("Greek salad", 15.50), ("Bean soup", 12.25), ("Pizza", 15.00), 
("Carbonara", 12.50), ("Kabasa", 17.00), ("Shawarma", 11.30);"""
logger.info("Inserting records into the tbl_courses table")
first_cursor.execute(insert_into_tbl_courses)
first_connection.commit()

In [21]:
execute_display_query_results(select_all_query("tbl_customers"))
print("\n")
execute_display_query_results(select_all_query("tbl_bookings"))
print("\n")
execute_display_query_results(select_all_query("tbl_courses"))

+------------+------------------+-------------+
| CustomerID |     FullName     | PhoneNumber |
+------------+------------------+-------------+
|     1      | Vanessa McCarthy |  757536378  |
|     2      |  Marcos Romero   |  757536379  |
|     3      |  Hiroki Yamane   |  757536376  |
|     4      |   Anna Iversen   |  757536375  |
|     5      |   Diana Pinto    |  757536374  |
|     6      |   Altay Ayhan    |  757636378  |
|     7      |   Jane Murphy    |  753536379  |
|     8      | Laurina Delgado  |  754536376  |
|     9      |   Mike Edwards   |  757236375  |
|     10     |  Karl Pederson   |  757936374  |
+------------+------------------+-------------+


+-----------+-------------+-------------+----------------+------------+
| BookingID | BookingDate | TableNumber | NumberOfGuests | CustomerID |
+-----------+-------------+-------------+----------------+------------+
|    10     | 2021-11-10  |      7      |       5        |     1      |
|    11     | 2021-11-10  |      5    

# Joins

## Task 1
Write an INNER JOIN SQL statement to combine the full name and the phone number of each customer from the Customers table with the related booking date and 'number of guests' from the Bookings table. 

In [22]:
inner_join_query = """SELECT FullName, PhoneNumber, BookingDate, NumberOfGuests FROM tbl_customers
INNER JOIN tbl_bookings ON tbl_customers.CustomerID = tbl_bookings.CustomerID;"""
execute_display_query_results(inner_join_query)

+------------------+-------------+-------------+----------------+
|     FullName     | PhoneNumber | BookingDate | NumberOfGuests |
+------------------+-------------+-------------+----------------+
| Vanessa McCarthy |  757536378  | 2021-11-10  |       5        |
| Vanessa McCarthy |  757536378  | 2021-11-11  |       5        |
|  Marcos Romero   |  757536379  | 2021-11-10  |       2        |
|  Marcos Romero   |  757536379  | 2021-11-12  |       2        |
|  Hiroki Yamane   |  757536376  | 2021-11-14  |       2        |
|   Anna Iversen   |  757536375  | 2021-11-10  |       2        |
|   Anna Iversen   |  757536375  | 2021-11-12  |       2        |
|   Anna Iversen   |  757536375  | 2021-11-14  |       2        |
|   Diana Pinto    |  757536374  | 2021-11-11  |       5        |
|   Altay Ayhan    |  757636378  | 2021-11-11  |       2        |
|   Altay Ayhan    |  757636378  | 2021-11-13  |       5        |
|   Jane Murphy    |  753536379  | 2021-11-11  |       2        |
+---------

## Task 2
Write a LEFT JOIN SQL statement to view the customer id from Customers table and the related booking id from the Bookings table.

In [23]:
left_join_query = """SELECT tbl_customers.CustomerID, BookingID FROM tbl_customers
LEFT JOIN tbl_bookings ON tbl_customers.CustomerID = tbl_bookings.CustomerID;"""
execute_display_query_results(left_join_query)

+------------+-----------+
| CustomerID | BookingID |
+------------+-----------+
|     7      |    15     |
|     8      |   None    |
|     9      |   None    |
|     5      |    13     |
|     4      |    12     |
|     4      |    18     |
|     4      |    21     |
|     3      |    20     |
|     1      |    10     |
|     1      |    16     |
|     2      |    11     |
|     2      |    17     |
|     6      |    14     |
|     6      |    19     |
|     10     |   None    |
+------------+-----------+


# Group By

In [24]:
create_new_order_table = """CREATE TABLE IF NOT EXISTS orders_tbl(OrderID INT, Department VARCHAR(100), OrderDate DATE, OrderQty INT, OrderTotal INT, PRIMARY KEY(OrderID));"""
first_cursor.execute(create_new_order_table)
print("Table orders_tbl successfully created")
logger.info("Table orders_tbl successfully created")

Table orders_tbl successfully created


In [25]:
insert_into_new_table = """INSERT INTO orders_tbl VALUES(1,'Lawn Care','2022-05-05',12,500),(2,'Decking','2022-05-22',150,1450),(3,'Compost and Stones','2022-05-27',20,780),(4,'Trees and Shrubs','2022-06-01',15,400),(5,'Garden Decor','2022-06-10',2,1250),
(6,'Lawn Care','2022-06-10',12,500),(7,'Decking','2022-06-25',150,1450),(8,'Compost and Stones','2022-05-29',20,780),(9,'Trees and Shrubs','2022-06-10',15,400),(10,'Garden Decor','2022-06-10',2,1250),(11,'Lawn Care','2022-06-25',10,400), 
(12,'Decking','2022-06-25',100,1400),(13,'Compost and Stones','2022-05-30',15,700),(14,'Trees and Shrubs','2022-06-15',10,300),(15,'Garden Decor','2022-06-11',2,1250),(16,'Lawn Care','2022-06-10',12,500),
(17,'Decking','2022-06-25',150,1450),(18,'Trees and Shrubs','2022-06-10',15,400),(19,'Lawn Care','2022-06-10',12,500),(20,'Decking','2022-06-25',150,1450),(21,'Decking','2022-06-25',150,1450);"""


logger.info("Inserting records into the orders_tbl table")
first_cursor.execute(insert_into_new_table)
first_connection.commit()

In [26]:
execute_display_query_results(select_all_query("orders_tbl"))

+---------+--------------------+------------+----------+------------+
| OrderID |     Department     | OrderDate  | OrderQty | OrderTotal |
+---------+--------------------+------------+----------+------------+
|    1    |     Lawn Care      | 2022-05-05 |    12    |    500     |
|    2    |      Decking       | 2022-05-22 |   150    |    1450    |
|    3    | Compost and Stones | 2022-05-27 |    20    |    780     |
|    4    |  Trees and Shrubs  | 2022-06-01 |    15    |    400     |
|    5    |    Garden Decor    | 2022-06-10 |    2     |    1250    |
|    6    |     Lawn Care      | 2022-06-10 |    12    |    500     |
|    7    |      Decking       | 2022-06-25 |   150    |    1450    |
|    8    | Compost and Stones | 2022-05-29 |    20    |    780     |
|    9    |  Trees and Shrubs  | 2022-06-10 |    15    |    400     |
|   10    |    Garden Decor    | 2022-06-10 |    2     |    1250    |
|   11    |     Lawn Care      | 2022-06-25 |    10    |    400     |
|   12    |      Dec

## Task 1
Write a SQL SELECT statement to group all records that have the same order date.

In [27]:
select_query = "SELECT OrderDate FROM orders_tbl GROUP BY OrderDate"
execute_display_query_results(select_query)

+------------+
| OrderDate  |
+------------+
| 2022-05-05 |
| 2022-05-22 |
| 2022-05-27 |
| 2022-06-01 |
| 2022-06-10 |
| 2022-06-25 |
| 2022-05-29 |
| 2022-05-30 |
| 2022-06-15 |
| 2022-06-11 |
+------------+


## Task 2
Write a SQL SELECT statement to retrieve the number of orders placed on the same day.

In [28]:
select_query = "SELECT OrderDate, Count(OrderQty) AS OrderQuantity FROM orders_tbl GROUP BY OrderDate"
execute_display_query_results(select_query)

+------------+---------------+
| OrderDate  | OrderQuantity |
+------------+---------------+
| 2022-05-05 |       1       |
| 2022-05-22 |       1       |
| 2022-05-27 |       1       |
| 2022-06-01 |       1       |
| 2022-06-10 |       7       |
| 2022-06-25 |       6       |
| 2022-05-29 |       1       |
| 2022-05-30 |       1       |
| 2022-06-15 |       1       |
| 2022-06-11 |       1       |
+------------+---------------+


## Task 3
Write a SQL SELECT statement to retrieve the total order quantities placed by each department.

In [29]:
select_query = "SELECT Department, Sum(OrderQty) AS OrderQuantity FROM orders_tbl GROUP BY Department"
execute_display_query_results(select_query)

+--------------------+---------------+
|     Department     | OrderQuantity |
+--------------------+---------------+
|     Lawn Care      |      58       |
|      Decking       |      850      |
| Compost and Stones |      55       |
|  Trees and Shrubs  |      55       |
|    Garden Decor    |       6       |
+--------------------+---------------+


## Task 4
Write a SQL SELECT statement to retrieve the number of orders placed on the same day between the following dates: 1st June 2022 and 30th June 2022.

In [30]:
select_query = "SELECT OrderDate, Count(OrderQty) AS OrderQuantity FROM orders_tbl WHERE OrderDate BETWEEN '2022-06-01' AND '2022-06-30' GROUP BY OrderDate"
execute_display_query_results(select_query)

+------------+---------------+
| OrderDate  | OrderQuantity |
+------------+---------------+
| 2022-06-01 |       1       |
| 2022-06-10 |       7       |
| 2022-06-25 |       6       |
| 2022-06-15 |       1       |
| 2022-06-11 |       1       |
+------------+---------------+


# Replace Into
How the REPLACE INTO statement works
The REPLACE statement checks whether the intended data record's unique key value already exists in the table before inserting it as a new record or updating it. 

The REPLACE INTO statement attempts to insert a new record or modify an existing record. In both cases, it checks whether the unique key of the proposed record already exists in the table. Suppose a value of NO or FALSE is returned. In that case, the REPLACE statement inserts the record similar to the INSERT INTO statement.

Suppose the key value already exists in the table (in other words, a duplicate key). In that case, the REPLACE statement deletes the existing record of data and replaces it with a new record of data. This happens regardless of whether you use the first or the second REPLACE statement syntax.

```sql
REPLACE INTO table_name (column1name, column2name, ...) 
VALUES (value1, value2, ...);
```


```sql
REPLACE INTO table_name SET column1name = value, column2name = value, ... ;
```

In [31]:
select_query = "SELECT * FROM tbl_orders"
execute_display_query_results(select_query)

+---------+----------+-----------+----------+---------+
| OrderID | ClientID | ProductID | Quantity |  Cost   |
+---------+----------+-----------+----------+---------+
|    1    |   Cl1    |    P1     |    10    | 500.00  |
|    2    |   Cl2    |    P2     |    5     | 100.00  |
|    3    |   Cl3    |    P3     |    20    | 800.00  |
|    4    |   Cl4    |    P4     |    15    | 150.00  |
|    5    |   Cl3    |    P3     |    10    | 450.00  |
|    6    |   Cl2    |    P2     |    5     | 800.00  |
|    7    |   Cl1    |    P4     |    22    | 1200.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00  |
|    9    |   Cl1    |    P1     |    10    | 500.00  |
|   10    |   Cl2    |    P2     |    5     | 100.00  |
+---------+----------+-----------+----------+---------+


## Task 1
Write a SQL REPLACE statement that inserts two new orders with the following details:

Order 11 data:
- OrderID = 11, 
- ClientID = "Cl2", 
- ProductID = "P3", 
- Quantity = 50, 
- Cost = 5000

Order 12 data:
- OrderID = 12, 
- ClientID = "Cl2", 
- ProductID = "P2", 
- Quantity = 25, 
- Cost = 1000

In [32]:
replace_query = """REPLACE INTO tbl_orders (OrderID, ClientID, ProductID, Quantity, Cost) 
VALUES (11, 'Cl2', 'P3', 50, 5000), (12, 'Cl2', 'P2', 25, 1000);"""

first_cursor.execute(replace_query)
first_connection.commit()

In [33]:
# select_query = "SELECT * FROM tbl_orders"
execute_display_query_results(select_all_query("tbl_orders"))

+---------+----------+-----------+----------+---------+
| OrderID | ClientID | ProductID | Quantity |  Cost   |
+---------+----------+-----------+----------+---------+
|    1    |   Cl1    |    P1     |    10    | 500.00  |
|    2    |   Cl2    |    P2     |    5     | 100.00  |
|    3    |   Cl3    |    P3     |    20    | 800.00  |
|    4    |   Cl4    |    P4     |    15    | 150.00  |
|    5    |   Cl3    |    P3     |    10    | 450.00  |
|    6    |   Cl2    |    P2     |    5     | 800.00  |
|    7    |   Cl1    |    P4     |    22    | 1200.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00  |
|    9    |   Cl1    |    P1     |    10    | 500.00  |
|   10    |   Cl2    |    P2     |    5     | 100.00  |
|   11    |   Cl2    |    P3     |    50    | 5000.00 |
|   12    |   Cl2    |    P2     |    25    | 1000.00 |
+---------+----------+-----------+----------+---------+


## Task 2
It was noticed that the cost of order number 11 is \\\$5000. This is a mistake. The order must cost \\$500. You must help them to change it to \\$500 by writing a relevant REPLACE statement. 

In [34]:
replace_query_set = """REPLACE INTO tbl_orders SET OrderID = 11, ClientID = 'Cl2', ProductID = 'P3', Quantity = 50, Cost = 500;"""
first_cursor.execute(replace_query_set)
first_connection.commit()

In [35]:
execute_display_query_results(select_all_query("tbl_orders"))

+---------+----------+-----------+----------+---------+
| OrderID | ClientID | ProductID | Quantity |  Cost   |
+---------+----------+-----------+----------+---------+
|    1    |   Cl1    |    P1     |    10    | 500.00  |
|    2    |   Cl2    |    P2     |    5     | 100.00  |
|    3    |   Cl3    |    P3     |    20    | 800.00  |
|    4    |   Cl4    |    P4     |    15    | 150.00  |
|    5    |   Cl3    |    P3     |    10    | 450.00  |
|    6    |   Cl2    |    P2     |    5     | 800.00  |
|    7    |   Cl1    |    P4     |    22    | 1200.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00  |
|    9    |   Cl1    |    P1     |    10    | 500.00  |
|   10    |   Cl2    |    P2     |    5     | 100.00  |
|   11    |   Cl2    |    P3     |    50    | 500.00  |
|   12    |   Cl2    |    P2     |    25    | 1000.00 |
+---------+----------+-----------+----------+---------+


# Subqueries

In [36]:
create_menu_items_table = """CREATE TABLE IF NOT EXISTS tbl_menuitems (ItemID INT, Name VARCHAR(200), Type VARCHAR(100), Price INT, PRIMARY KEY (ItemID));"""
first_cursor.execute(create_menu_items_table)

create_menu_table = """CREATE TABLE IF NOT EXISTS tbl_menu (MenuID INT, ItemID INT, Cuisine VARCHAR(100), PRIMARY KEY (MenuID, ItemID));"""
first_cursor.execute(create_menu_table)

create_bookings_table = """CREATE TABLE IF NOT EXISTS bookings_tbl (BookingID INT, TableNo INT, GuestFirstName VARCHAR(100), GuestLastName VARCHAR(100), BookingSlot TIME, EmployeeID INT, PRIMARY KEY (BookingID));"""
first_cursor.execute(create_bookings_table)

create_table_orders_table = """CREATE TABLE IF NOT EXISTS tbl_tableorders (OrderID INT, TableNo INT, MenuID INT, BookingID INT, BillAmount INT, Quantity INT, PRIMARY KEY (OrderID,TableNo));"""
first_cursor.execute(create_table_orders_table)

In [37]:
insert_into_menuitems_query = """INSERT INTO tbl_menuitems VALUES
(1,'Olives','Starters', 5), (2,'Flatbread','Starters', 5),(3, 'Minestrone', 'Starters', 8), (4, 'Tomato bread','Starters', 8), (5, 'Falafel', 'Starters', 7), 
(6, 'Hummus', 'Starters', 5), (7, 'Greek salad', 'Main Courses', 15), (8, 'Bean soup', 'Main Courses', 12), (9, 'Pizza', 'Main Courses', 15), 
(10,'Greek yoghurt','Desserts', 7), (11, 'Ice cream', 'Desserts', 6),(12, 'Cheesecake', 'Desserts', 4), 
(13, 'Athens White wine', 'Drinks', 25), (14, 'Corfu Red Wine', 'Drinks', 30), (15, 'Turkish Coffee', 'Drinks', 10), 
(16, 'Turkish Coffee', 'Drinks', 10), (17, 'Kabasa', 'Main Courses', 17);"""
first_cursor.execute(insert_into_menuitems_query)
first_connection.commit()


insert_into_menu_query = """INSERT INTO tbl_menu VALUES(1, 1, 'Greek'), (1, 7, 'Greek'), (1, 10, 'Greek'), (1, 13, 'Greek'), (2, 3, 'Italian'), (2, 9, 'Italian'), (2, 12, 'Italian'), 
(2, 15, 'Italian'), (3, 5, 'Turkish'), (3, 17, 'Turkish'), (3, 11, 'Turkish'), (3, 16, 'Turkish');"""
first_cursor.execute(insert_into_menu_query)
first_connection.commit()


insert_into_bookings_query = """INSERT INTO bookings_tbl VALUES(1,12,'Anna','Iversen','19:00:00',1),  
(2, 12, 'Joakim', 'Iversen', '19:00:00', 1), (3, 19, 'Vanessa', 'McCarthy', '15:00:00', 3), 
(4, 15, 'Marcos', 'Romero', '17:30:00', 4), (5, 5, 'Hiroki', 'Yamane', '18:30:00', 2),
(6, 8, 'Diana', 'Pinto', '20:00:00', 5);"""
first_cursor.execute(insert_into_bookings_query)
first_connection.commit()


insert_into_tableorders_query = """INSERT INTO tbl_tableorders VALUES(1, 12, 1, 1, 2, 86), (2, 19, 2, 2, 1, 37), (3, 15, 2, 3, 1, 37), (4, 5, 3, 4, 1, 40), (5, 8, 1, 5, 1, 43);"""
first_cursor.execute(insert_into_tableorders_query)
first_connection.commit()

In [38]:
execute_display_query_results(select_all_query("tbl_menuitems"))

+--------+-------------------+--------------+-------+
| ItemID |       Name        |     Type     | Price |
+--------+-------------------+--------------+-------+
|   1    |      Olives       |   Starters   |   5   |
|   2    |     Flatbread     |   Starters   |   5   |
|   3    |    Minestrone     |   Starters   |   8   |
|   4    |   Tomato bread    |   Starters   |   8   |
|   5    |      Falafel      |   Starters   |   7   |
|   6    |      Hummus       |   Starters   |   5   |
|   7    |    Greek salad    | Main Courses |  15   |
|   8    |     Bean soup     | Main Courses |  12   |
|   9    |       Pizza       | Main Courses |  15   |
|   10   |   Greek yoghurt   |   Desserts   |   7   |
|   11   |     Ice cream     |   Desserts   |   6   |
|   12   |    Cheesecake     |   Desserts   |   4   |
|   13   | Athens White wine |    Drinks    |  25   |
|   14   |  Corfu Red Wine   |    Drinks    |  30   |
|   15   |  Turkish Coffee   |    Drinks    |  10   |
|   16   |  Turkish Coffee  

In [39]:
execute_display_query_results(select_all_query("tbl_menu"))

+--------+--------+---------+
| MenuID | ItemID | Cuisine |
+--------+--------+---------+
|   1    |   1    |  Greek  |
|   1    |   7    |  Greek  |
|   1    |   10   |  Greek  |
|   1    |   13   |  Greek  |
|   2    |   3    | Italian |
|   2    |   9    | Italian |
|   2    |   12   | Italian |
|   2    |   15   | Italian |
|   3    |   5    | Turkish |
|   3    |   11   | Turkish |
|   3    |   16   | Turkish |
|   3    |   17   | Turkish |
+--------+--------+---------+


In [40]:
execute_display_query_results(select_all_query("bookings_tbl"))

+-----------+---------+----------------+---------------+-------------+------------+
| BookingID | TableNo | GuestFirstName | GuestLastName | BookingSlot | EmployeeID |
+-----------+---------+----------------+---------------+-------------+------------+
|     1     |   12    |      Anna      |    Iversen    |  19:00:00   |     1      |
|     2     |   12    |     Joakim     |    Iversen    |  19:00:00   |     1      |
|     3     |   19    |    Vanessa     |   McCarthy    |  15:00:00   |     3      |
|     4     |   15    |     Marcos     |    Romero     |  17:30:00   |     4      |
|     5     |    5    |     Hiroki     |    Yamane     |  18:30:00   |     2      |
|     6     |    8    |     Diana      |     Pinto     |  20:00:00   |     5      |
+-----------+---------+----------------+---------------+-------------+------------+


In [41]:
execute_display_query_results(select_all_query("tbl_tableorders"))

+---------+---------+--------+-----------+------------+----------+
| OrderID | TableNo | MenuID | BookingID | BillAmount | Quantity |
+---------+---------+--------+-----------+------------+----------+
|    1    |   12    |   1    |     1     |     2      |    86    |
|    2    |   19    |   2    |     2     |     1      |    37    |
|    3    |   15    |   2    |     3     |     1      |    37    |
|    4    |    5    |   3    |     4     |     1      |    40    |
|    5    |    8    |   1    |     5     |     1      |    43    |
+---------+---------+--------+-----------+------------+----------+


## Task 1
Write a SQL SELECT query to find all bookings that are due after the booking of the guest ‘Vanessa McCarthy’.

In [42]:
select_query = """SELECT * FROM bookings_tbl WHERE BookingSlot > ANY (SELECT BookingSlot FROM bookings_tbl WHERE GuestFirstName = 'Vanessa' AND GuestLastName = 'McCarthy');"""
execute_display_query_results(select_query)

+-----------+---------+----------------+---------------+-------------+------------+
| BookingID | TableNo | GuestFirstName | GuestLastName | BookingSlot | EmployeeID |
+-----------+---------+----------------+---------------+-------------+------------+
|     1     |   12    |      Anna      |    Iversen    |  19:00:00   |     1      |
|     2     |   12    |     Joakim     |    Iversen    |  19:00:00   |     1      |
|     4     |   15    |     Marcos     |    Romero     |  17:30:00   |     4      |
|     5     |    5    |     Hiroki     |    Yamane     |  18:30:00   |     2      |
|     6     |    8    |     Diana      |     Pinto     |  20:00:00   |     5      |
+-----------+---------+----------------+---------------+-------------+------------+


## Task 2
Write a SQL SELECT query to find the menu items that are more expensive than all the 'Starters' and 'Desserts' menu item types.

In [43]:
select_query = """SELECT * FROM tbl_menuitems WHERE Price > ALL (SELECT Price FROM tbl_menuitems WHERE Type IN ('Starters', 'Desserts'));"""
execute_display_query_results(select_query)

+--------+-------------------+--------------+-------+
| ItemID |       Name        |     Type     | Price |
+--------+-------------------+--------------+-------+
|   7    |    Greek salad    | Main Courses |  15   |
|   8    |     Bean soup     | Main Courses |  12   |
|   9    |       Pizza       | Main Courses |  15   |
|   13   | Athens White wine |    Drinks    |  25   |
|   14   |  Corfu Red Wine   |    Drinks    |  30   |
|   15   |  Turkish Coffee   |    Drinks    |  10   |
|   16   |  Turkish Coffee   |    Drinks    |  10   |
|   17   |      Kabasa       | Main Courses |  17   |
+--------+-------------------+--------------+-------+


## Task 3
Write a SQL SELECT query to find the menu items that costs the same as the starter menu items that are Italian cuisine.

In [44]:
select_query = """SELECT * FROM tbl_menuitems WHERE Price = (SELECT Price FROM tbl_menuitems INNER JOIN tbl_menu ON tbl_menuitems.ItemID = tbl_menu.ItemID WHERE tbl_menu.Cuisine = 'Italian' AND tbl_menuitems.Type = 'Starters');"""
execute_display_query_results(select_query)

+--------+--------------+----------+-------+
| ItemID |     Name     |   Type   | Price |
+--------+--------------+----------+-------+
|   3    |  Minestrone  | Starters |   8   |
|   4    | Tomato bread | Starters |   8   |
+--------+--------------+----------+-------+


## Task 4
Write a SQL SELECT query to find the menu items that were not ordered by the guests who placed bookings.

In [45]:
select_query = """SELECT * FROM tbl_menuitems WHERE ItemID != ALL (SELECT ItemID FROM tbl_menu INNER JOIN tbl_tableorders ON tbl_menu.MenuID = tbl_tableorders.MenuID);"""
execute_display_query_results(select_query)

+--------+----------------+--------------+-------+
| ItemID |      Name      |     Type     | Price |
+--------+----------------+--------------+-------+
|   2    |   Flatbread    |   Starters   |   5   |
|   4    |  Tomato bread  |   Starters   |   8   |
|   6    |     Hummus     |   Starters   |   5   |
|   8    |   Bean soup    | Main Courses |  12   |
|   14   | Corfu Red Wine |    Drinks    |  30   |
+--------+----------------+--------------+-------+


# Views
Virtual tables used for accessing and manipulatin data with MySQL

In [46]:
execute_display_query_results(select_all_query("tbl_orders"))

+---------+----------+-----------+----------+---------+
| OrderID | ClientID | ProductID | Quantity |  Cost   |
+---------+----------+-----------+----------+---------+
|    1    |   Cl1    |    P1     |    10    | 500.00  |
|    2    |   Cl2    |    P2     |    5     | 100.00  |
|    3    |   Cl3    |    P3     |    20    | 800.00  |
|    4    |   Cl4    |    P4     |    15    | 150.00  |
|    5    |   Cl3    |    P3     |    10    | 450.00  |
|    6    |   Cl2    |    P2     |    5     | 800.00  |
|    7    |   Cl1    |    P4     |    22    | 1200.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00  |
|    9    |   Cl1    |    P1     |    10    | 500.00  |
|   10    |   Cl2    |    P2     |    5     | 100.00  |
|   11    |   Cl2    |    P3     |    50    | 500.00  |
|   12    |   Cl2    |    P2     |    25    | 1000.00 |
+---------+----------+-----------+----------+---------+


## Task 1
Write a SQL statement to create the OrdersView Virtual table based on the Orders table. The table must include the following columns: Order ID, Quantity and Cost. Once you have executed the query, select all data from the OrdersView table. 

In [47]:
create_view_query = """CREATE VIEW OrdersView AS SELECT OrderID, Quantity, Cost FROM tbl_orders;"""
first_cursor.execute(create_view_query)

In [48]:
select_view_query = """SELECT * FROM OrdersView;"""
execute_display_query_results(select_view_query)

+---------+----------+---------+
| OrderID | Quantity |  Cost   |
+---------+----------+---------+
|    1    |    10    | 500.00  |
|    2    |    5     | 100.00  |
|    3    |    20    | 800.00  |
|    4    |    15    | 150.00  |
|    5    |    10    | 450.00  |
|    6    |    5     | 800.00  |
|    7    |    22    | 1200.00 |
|    8    |    15    | 150.00  |
|    9    |    10    | 500.00  |
|   10    |    5     | 100.00  |
|   11    |    50    | 500.00  |
|   12    |    25    | 1000.00 |
+---------+----------+---------+


## Task 2
Write a SQL statement that utilizes the ‘OrdersView’ virtual table to Update the base Orders table. In the UPDATE TABLE statement, change the cost to 200 where the order id equals 2. Once you have executed the query, select all data from the OrdersView table. 

In [49]:
update_orders_view_query = """UPDATE OrdersView SET Cost = 200 WHERE OrderID = 2;"""
first_cursor.execute(update_orders_view_query)

In [50]:
select_view_query = """SELECT * FROM OrdersView;"""
execute_display_query_results(select_view_query)

+---------+----------+---------+
| OrderID | Quantity |  Cost   |
+---------+----------+---------+
|    1    |    10    | 500.00  |
|    2    |    5     | 200.00  |
|    3    |    20    | 800.00  |
|    4    |    15    | 150.00  |
|    5    |    10    | 450.00  |
|    6    |    5     | 800.00  |
|    7    |    22    | 1200.00 |
|    8    |    15    | 150.00  |
|    9    |    10    | 500.00  |
|   10    |    5     | 100.00  |
|   11    |    50    | 500.00  |
|   12    |    25    | 1000.00 |
+---------+----------+---------+


In [51]:
execute_display_query_results(select_all_query("tbl_orders"))

+---------+----------+-----------+----------+---------+
| OrderID | ClientID | ProductID | Quantity |  Cost   |
+---------+----------+-----------+----------+---------+
|    1    |   Cl1    |    P1     |    10    | 500.00  |
|    2    |   Cl2    |    P2     |    5     | 200.00  |
|    3    |   Cl3    |    P3     |    20    | 800.00  |
|    4    |   Cl4    |    P4     |    15    | 150.00  |
|    5    |   Cl3    |    P3     |    10    | 450.00  |
|    6    |   Cl2    |    P2     |    5     | 800.00  |
|    7    |   Cl1    |    P4     |    22    | 1200.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00  |
|    9    |   Cl1    |    P1     |    10    | 500.00  |
|   10    |   Cl2    |    P2     |    5     | 100.00  |
|   11    |   Cl2    |    P3     |    50    | 500.00  |
|   12    |   Cl2    |    P2     |    25    | 1000.00 |
+---------+----------+-----------+----------+---------+


## Task 3
Write a SQL statement that changes the name of the ‘OrdersView’ virtual table to ClientsOrdersView.

In [52]:
rename_view_query = """RENAME TABLE OrdersView TO ClientsOrdersView;"""
first_cursor.execute(rename_view_query)

In [53]:
select_view_query = """SELECT * FROM ClientsOrdersView;"""
execute_display_query_results(select_view_query)

+---------+----------+---------+
| OrderID | Quantity |  Cost   |
+---------+----------+---------+
|    1    |    10    | 500.00  |
|    2    |    5     | 200.00  |
|    3    |    20    | 800.00  |
|    4    |    15    | 150.00  |
|    5    |    10    | 450.00  |
|    6    |    5     | 800.00  |
|    7    |    22    | 1200.00 |
|    8    |    15    | 150.00  |
|    9    |    10    | 500.00  |
|   10    |    5     | 100.00  |
|   11    |    50    | 500.00  |
|   12    |    25    | 1000.00 |
+---------+----------+---------+


## Task 4
Write a SQL statement to delete the Orders virtual table.

In [54]:
drop_view_query = """DROP VIEW ClientsOrdersView;"""
first_cursor.execute(drop_view_query)

# Procedures

## Task 1 
Write a SQL statement that creates a stored procedure called 'GetOrdersData' which retrieves all data from the Orders table.

In [55]:
create_procedure_query = """CREATE PROCEDURE GetOrdersData() SELECT * FROM tbl_orders;"""
first_cursor.execute(create_procedure_query)

In [56]:
first_cursor.callproc("GetOrdersData")

results=next(first_cursor.stored_results())
table_column_names = results.column_names
dataset = results.fetchall()

execute_display_query_results(table_column_names=table_column_names, results=dataset)

+---------+----------+-----------+----------+---------+
| OrderID | ClientID | ProductID | Quantity |  Cost   |
+---------+----------+-----------+----------+---------+
|    1    |   Cl1    |    P1     |    10    | 500.00  |
|    2    |   Cl2    |    P2     |    5     | 200.00  |
|    3    |   Cl3    |    P3     |    20    | 800.00  |
|    4    |   Cl4    |    P4     |    15    | 150.00  |
|    5    |   Cl3    |    P3     |    10    | 450.00  |
|    6    |   Cl2    |    P2     |    5     | 800.00  |
|    7    |   Cl1    |    P4     |    22    | 1200.00 |
|    8    |   Cl3    |    P1     |    15    | 150.00  |
|    9    |   Cl1    |    P1     |    10    | 500.00  |
|   10    |   Cl2    |    P2     |    5     | 100.00  |
|   11    |   Cl2    |    P3     |    50    | 500.00  |
|   12    |   Cl2    |    P2     |    25    | 1000.00 |
+---------+----------+-----------+----------+---------+


# Project

## Task 1: Filter data using the WHERE clause and logical operators.  
Create SQL statement to print all records from tbl_bookings table for the following bookings dates using the BETWEEN operator: 2021-11-11, 2021-11-12 and 2021-11-13. 

In [57]:
select_query = """SELECT * FROM tbl_bookings WHERE BookingDate BETWEEN '2021-11-11' AND '2021-11-13'"""
execute_display_query_results(select_query)

+-----------+-------------+-------------+----------------+------------+
| BookingID | BookingDate | TableNumber | NumberOfGuests | CustomerID |
+-----------+-------------+-------------+----------------+------------+
|    13     | 2021-11-11  |      2      |       5        |     5      |
|    14     | 2021-11-11  |      5      |       2        |     6      |
|    15     | 2021-11-11  |      3      |       2        |     7      |
|    16     | 2021-11-11  |      3      |       5        |     1      |
|    17     | 2021-11-12  |      5      |       2        |     2      |
|    18     | 2021-11-12  |      3      |       2        |     4      |
|    19     | 2021-11-13  |      7      |       5        |     6      |
+-----------+-------------+-------------+----------------+------------+


## Task 2: Create a JOIN query.  
Create a JOIN SQL statement on the Customers and Bookings tables. The statement must print the customers full names and related bookings IDs from the date 2021-11-11.

In [58]:
join_query = """SELECT FullName, BookingID, NumberOfGuests FROM tbl_customers
INNER JOIN tbl_bookings ON tbl_customers.CustomerID = tbl_bookings.CustomerID WHERE BookingDate = '2021-11-11';"""
execute_display_query_results(join_query)

+------------------+-----------+----------------+
|     FullName     | BookingID | NumberOfGuests |
+------------------+-----------+----------------+
|   Diana Pinto    |    13     |       5        |
|   Altay Ayhan    |    14     |       2        |
|   Jane Murphy    |    15     |       2        |
| Vanessa McCarthy |    16     |       5        |
+------------------+-----------+----------------+


## Task 3: Create a GROUP BY query.  
Create a SQL statement to print the bookings dates from Bookings table. The statement must show the total number of bookings placed on each of the printed dates using the GROUP BY BookingDate. 

In [59]:
groupby_query: str = "SELECT BookingDate, COUNT(BookingDate) AS NumberOfBookings FROM tbl_bookings GROUP BY BookingDate"
execute_display_query_results(groupby_query)

+-------------+------------------+
| BookingDate | NumberOfBookings |
+-------------+------------------+
| 2021-11-10  |        3         |
| 2021-11-11  |        4         |
| 2021-11-12  |        2         |
| 2021-11-13  |        1         |
| 2021-11-14  |        2         |
+-------------+------------------+


## Task 4: Create a REPLACE statement.  
Create a SQL REPLACE statement that updates the cost of the Kabsa course from \\$17.00 to \\$20.00.

In [60]:
replace_query_set = """REPLACE INTO tbl_courses SET CourseName = 'Kabasa', Cost = '20.50';"""
first_cursor.execute(replace_query_set)
first_connection.commit()

In [61]:
# select_query = "SELECT * FROM tbl_courses"
execute_display_query_results(select_all_query("tbl_courses"))

+-------------+-------+
| CourseName  | Cost  |
+-------------+-------+
|  Bean soup  | 12.25 |
|  Carbonara  | 12.50 |
| Greek salad | 15.50 |
|   Kabasa    | 20.50 |
|    Pizza    | 15.00 |
|  Shawarma   | 11.30 |
+-------------+-------+


## Task 5: Create constraints

Create a new table called "DeliveryAddress" in the Little Lemon database with the following columns and constraints:  
- ID: INT PRIMARY KEY
- Address: VARCHAR(255) NOT NULL
- Type: NOT NULL DEFAULT "Private"
- CustomerID: INT NOT NULL FOREIGN KEY referencing CustomerID in the Customers table

In [62]:
create_table_deliveryaddress_query = """CREATE TABLE IF NOT EXISTS tbl_delivery_address (ID INT NOT NULL PRIMARY KEY, Address VARCHAR(255) NOT NULL, TYPE VARCHAR(100) NOT NULL DEFAULT "PRIVATE",  
CustomerID INT NOT NULL, FOREIGN KEY (CustomerID) REFERENCES tbl_customers (CustomerID) ON DELETE CASCADE ON UPDATE CASCADE);"""

first_cursor.execute(create_table_deliveryaddress_query)

In [63]:
show_query = "SHOW COLUMNS FROM tbl_delivery_address"
execute_display_query_results(show_query)

+------------+--------------+------+-----+---------+-------+
|   Field    |     Type     | Null | Key | Default | Extra |
+------------+--------------+------+-----+---------+-------+
|     ID     |     int      |  NO  | PRI |   NULL  |  NULL |
|  Address   | varchar(255) |  NO  | NULL|   NULL  |  NULL |
|    TYPE    | varchar(100) |  NO  | NULL| PRIVATE |  NULL |
| CustomerID |     int      |  NO  | MUL |   NULL  |  NULL |
+------------+--------------+------+-----+---------+-------+


## Task 6: Alter table structure
Create a SQL statement that adds a new column called 'Ingredients' to the Courses table.
- Ingredients: VARCHAR(255)

In [64]:
alter_table_query = """ALTER TABLE tbl_courses ADD COLUMN Ingredients VARCHAR(255);"""
first_cursor.execute(alter_table_query)

In [65]:
show_query = "SHOW COLUMNS FROM tbl_courses"
execute_display_query_results(show_query)

+-------------+--------------+------+-----+---------+-------+
|    Field    |     Type     | Null | Key | Default | Extra |
+-------------+--------------+------+-----+---------+-------+
| CourseName  | varchar(255) |  NO  | PRI |   NULL  |  NULL |
|    Cost     | decimal(4,2) | YES  | NULL|   NULL  |  NULL |
| Ingredients | varchar(255) | YES  | NULL|   NULL  |  NULL |
+-------------+--------------+------+-----+---------+-------+


## Task 7: Create a subquery  
Create a SQL statement with a subquery that prints the full names of all customers who made bookings in the restaurant on the following date: 2021-11-11.

In [66]:
subquery_query: str = """SELECT FullName FROM tbl_customers WHERE CustomerID = ANY (SELECT CustomerID FROM tbl_bookings WHERE BookingDate = '2021-11-11');"""
execute_display_query_results(subquery_query)

+------------------+
|     FullName     |
+------------------+
|   Diana Pinto    |
|   Altay Ayhan    |
|   Jane Murphy    |
| Vanessa McCarthy |
+------------------+


## Task 8: Create a virtual table  
Create the "BookingsView" virtual table to print all bookings IDs, bookings dates and the number of guests for bookings made in the restaurant before 2021-11-13 and where number of guests is larger than 3.

In [67]:
create_view_query: str = """CREATE VIEW BookingsView AS SELECT BookingID, BookingDate, NumberOfGuests FROM tbl_bookings WHERE BookingDate < '2021-11-13' AND NumberOfGuests > 3;"""
first_cursor.execute(create_view_query)

select_query = """SELECT * FROM BookingsView;"""
execute_display_query_results(select_query)

+-----------+-------------+----------------+
| BookingID | BookingDate | NumberOfGuests |
+-----------+-------------+----------------+
|    10     | 2021-11-10  |       5        |
|    13     | 2021-11-11  |       5        |
|    16     | 2021-11-11  |       5        |
+-----------+-------------+----------------+


## Task 9: Create a stored procedure

Create a stored procedure called 'GetBookingsData'. The procedure must contain one date parameter called "InputDate". This parameter retrieves all data from the Bookings table based on the user input of the date.

In [68]:
stored_procedure_query = """
CREATE PROCEDURE GetBookingsData(IN inputDate DATE)
BEGIN
SELECT BookingID, BookingDate, TableNumber, NumberOfGuests, tbl_bookings.CustomerID
FROM tbl_bookings
INNER JOIN tbl_customers ON tbl_bookings.CustomerID = tbl_customers.CustomerID
WHERE BookingDate = inputDate;
END
"""
first_cursor.execute(stored_procedure_query)

In [69]:
first_cursor.callproc("GetBookingsData", ('2021-11-13', ))
logger.info("Called the GetBookingsData procedure with parameters: ('2021-11-13', )")
results=next(first_cursor.stored_results())
table_column_names = results.column_names
dataset = results.fetchall()

execute_display_query_results(table_column_names=table_column_names, results=dataset)

+-----------+-------------+-------------+----------------+------------+
| BookingID | BookingDate | TableNumber | NumberOfGuests | CustomerID |
+-----------+-------------+-------------+----------------+------------+
|    19     | 2021-11-13  |      7      |       5        |     6      |
+-----------+-------------+-------------+----------------+------------+


## Task 10: Use the String function

Create a SQL SELECT query using appropriate MySQL string function to list "Booking Details" including booking ID, booking date and number of guests. The data must be listed in the same format as the following example:

- ID: 10, 
- Date: 2021-11-10, 
- Number of guests: 5

In [70]:
select_query = """SELECT CONCAT("ID: ", BookingID, ", Date: ", BookingDate, ", Number of guests: ", NumberOfGuests) AS 'Booking Details' FROM tbl_bookings;"""
execute_display_query_results(select_query)

+-----------------------------------------------+
|                Booking Details                |
+-----------------------------------------------+
| ID: 10, Date: 2021-11-10, Number of guests: 5 |
| ID: 11, Date: 2021-11-10, Number of guests: 2 |
| ID: 12, Date: 2021-11-10, Number of guests: 2 |
| ID: 13, Date: 2021-11-11, Number of guests: 5 |
| ID: 14, Date: 2021-11-11, Number of guests: 2 |
| ID: 15, Date: 2021-11-11, Number of guests: 2 |
| ID: 16, Date: 2021-11-11, Number of guests: 5 |
| ID: 17, Date: 2021-11-12, Number of guests: 2 |
| ID: 18, Date: 2021-11-12, Number of guests: 2 |
| ID: 19, Date: 2021-11-13, Number of guests: 5 |
| ID: 20, Date: 2021-11-14, Number of guests: 2 |
| ID: 21, Date: 2021-11-14, Number of guests: 2 |
+-----------------------------------------------+


In [71]:
first_cursor.close()
first_connection.close()

I enjoyed building a database schema from scratch and writing SQL queries to retrieve meaningful insights from data.
Seeing how data can be structured and manipulated was exciting.

I developed strong skills in SQL, including query writing, creating tables, and database normalization. Additionally, 
I improved my problem-solving abilities related to relational databases.
